In [0]:
# !pip install tensorflow==2.0.0-alpha0
# %reload_ext tensorboard.notebook

In [0]:
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torchnlp.nn as nlp_nn
import pandas as pd
import numpy as np
import string
import json
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.animation as animation
torch.manual_seed(1)
# from tensorflow import summary
import random
from torch.autograd import Variable
# import tensorflow as tf

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
!pip install torchviz
from torchviz import make_dot

In [0]:
# a = []
# while(1):
#     a.append('1')

In [0]:
# current_time = str(datetime.datetime.now().timestamp())
# train_log_dir = 'logs/tensorboard/train/' + current_time
# test_log_dir = 'logs/tensorboard/test/' + current_time
# train_summary_writer = summary.create_file_writer(train_log_dir)
# test_summary_writer = summary.create_file_writer(test_log_dir)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
class LSTM_TE(nn.Module):
    def __init__(self, embedding_dim,hidden_dim, vocab_size, class_numbers):
        super(LSTM_TE, self).__init__()
        self.H = hidden_dim
        self.K = class_numbers
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.H)
        self.att = nn.Linear(self.H, self.K,bias = True)
        self.att_norm = nn.Softmax(dim = 0)
        self.act = nn.Sigmoid()
        self.fin_ = nn.Softmax(dim = 0)
        self.fcs = [ nn.Linear(self.H,1) for _ in range(self.K)]
        self.c_prob = [ nn.Sigmoid() for _ in range(self.K)]
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_out = lstm_out.view(len(sentence),self.H)
        att_denorm = self.att(lstm_out.view(len(sentence), -1)) # a_t = W_att * h_t + b_att
        att_denorm = self.act(att_denorm)       # a_t = sigmoid(a_t)
        As = self.att_norm(att_denorm) # A = softmax_row({a_1,..,a_d}^T)
        As = As.view(len(sentence),self.K) # A = D*K
        T = torch.zeros((self.H,self.K))
        for i in range(self.H):
          T[i] = torch.mul(torch.transpose(As,0,1),torch.transpose(lstm_out,0,1)[i]).sum(dim=1)
        # tag_scores  = torch.mul(As,lstm_out) # produce T = {T1,..,TK} 
        # tag_scores = self.fin_(tag_scores.sum(dim = 0)) 
        T = torch.transpose(T,0,1)

        rt = torch.zeros((self.K,1)).to(device)
        for i in range(self.K):
          rt[i] = self.fcs[i](T[i])
          rt[i] = self.c_prob[i](rt[i])
        return  rt
    def init_hidden(self,batch_size):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        weights = next(self.parameters()).data
        h = (weights.new(1, batch_size, self.hidden_dim).zero_().to(device),
             weights.new(1, batch_size, self.hidden_dim).zero_().to(device))
        return h

In [0]:
# model = LSTM_TE(5, 3, 4000, 8).to(device) 

In [0]:
# inp = [ random.randint(1,30) for _ in range(5)]
# inp = torch.tensor(inp,dtype = torch.long).to(device)
# L  = model(inp)

In [0]:
# print(L)
# # print(a.shape,b.shape)
# # torch.mul(torch.transpose(a,0,1),torch.transpose(b,0,1)[0]).sum(dim = 1)

In [0]:
# make_dot(L)

In [0]:
# parse the data
# load drive
import json
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# load vocab
root_path = '/content/gdrive/My Drive/RDoCTask2019TrainTestData/RDoCTask1TrainData'
with open(root_path+"/vocab","r") as f:
  vocab = json.load(f)
  f.close()
df =pd.read_csv(root_path+"/train_csv")
root_path2 = '/content/gdrive/My Drive/RDoCTask2019TrainTestData/RDoCTask1TestData'
df2 =pd.read_csv(root_path+"/test_csv")

In [0]:
def word_to_idx(w):
  return vocab(w)
def sent_to_idxs(txt):
  lx = len(vocab.keys())
  return [ vocab[w] if w in vocab.keys() else lx for w in txt]

In [0]:
def listStr_lst(x):
  if type(x) != list:
    x = x[1:-1].split(",")
    x = list(map(lambda y: y.strip()[1:-1],x))
  return x
df.title = df.title.apply(lambda x: listStr_lst(x))
df.abstract = df.abstract.apply(lambda x: listStr_lst(x))
df['inp'] = df.title + df.abstract

In [0]:
df['inp'] = df['inp'].apply( lambda x : sent_to_idxs(x))

In [0]:
df2.title = df2.title.apply(lambda x: listStr_lst(x))
df2.abstract = df2.abstract.apply(lambda x: listStr_lst(x))
df2['inp'] = df2.title + df2.abstract
df2['inp'] = df2['inp'].apply( lambda x : sent_to_idxs(x))

In [0]:
df2.columns =['pmid', 'title', 'abstract', 'y_belong',
       'Relevant Context', 'label', 'inp']

In [0]:
VOCAB_SIZE = len(vocab.keys()) + 2 # indexing from 1 and 1 extra for unknown words
CLASS_SIZE = 8
EMBED_DIM = 128
HIDDEN_DIM = 32

In [0]:
# class LSTMTagger(nn.Module):

#     def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
#         super(LSTMTagger, self).__init__()
#         self.hidden_dim = hidden_dim

#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

#         # The LSTM takes word embeddings as inputs, and outputs hidden states
#         # with dimensionality hidden_dim.
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim)

#         # The linear layer that maps from hidden state space to tag space
#         self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

#     def forward(self, sentence):
#         embeds = self.word_embeddings(sentence)
#         lstm_out, h = self.lstm(embeds.view(len(sentence), 1, -1))
#         lstm_out = lstm_out.contiguous().view(-1,self.hidden_dim)
#         tag_space = self.hidden2tag(lstm_out)
#         tag_scores = F.log_softmax(tag_space, dim=1)
#         return tag_scores[-1]
#     def init_hidden(self,batch_size):
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#         weights = next(self.parameters()).data
#         h = (weights.new(1, batch_size, self.hidden_dim).zero_().to(device),
#              weights.new(1, batch_size, self.hidden_dim).zero_().to(device))
        
#         return h

In [0]:
def toTensor(inp):
  return torch.tensor(inp,dtype = torch.long).to(device)

In [0]:
df = df.sample(frac=1)
df2 = df2.sample(frac=1)

In [0]:
X_train = df['inp'].apply(toTensor).values
Y_train = df['label'].apply(toTensor).values


In [0]:
X_test = df2['inp'].apply(toTensor).values
Y_test = df2['label'].values
Y_belongs = df2['y_belong']

In [0]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
# inp = torch.tensor(df['inp'].head(1)[0],dtype = torch.long).to(device)
# a = model(inp)
# # make_dot(a)

In [0]:
# %tensorboard --logdir logs/tensorboard

In [0]:
# model = model.to(device)

In [0]:
model = LSTM_TE(EMBED_DIM, HIDDEN_DIM, VOCAB_SIZE, CLASS_SIZE).to(device)
loss_function = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer = optim.Adagrad(model.parameters())#,weight_decay=1e-5)
# optimizer = optim.SGD(model.parameters(), lr=0.1)

In [0]:
loss_list = []
j = 0
for epoch in range(60):
  epoch_loss = 0 
  for i in range(len(X_train)):
    j += 1
    x,y = X_train[i].to(device),Y_train[i].to(device)
    model.zero_grad()
    tg_score = model(x)
    tg_score = tg_score.reshape((1,8))
    y = y.reshape((1))
    # print(tg_score)
    # print(y)
    loss = loss_function(tg_score,y)
    epoch_loss += loss.item()
    loss.backward()
    optimizer.step()
  # with train_summary_writer.as_default():
  #       summary.scalar('loss', epoch_loss/len(X_train), step=epoch)
  # if ( epoch % 10 == 0):
  loss_list.append(epoch_loss/len(X_train))
  # def animate(i):
  #   ax1.clear()
  #   ax1.plot(range(epoch+1),loss_list)
  # ani = animation.FuncAnimation(fig, animate, interval=1000)
  # plt.show()
  print("Epoch [%d] %.6f "%(epoch,epoch_loss/len(X_train)))
  epoch_loss = 0

In [0]:
plt.plot(range(len(loss_list)),loss_list)
plt.show()

In [0]:
# y_pred = torch.randn((len(X_test),8)).to(device)
# for i in range(len(X_test)):
#   print(model(X_test[i].to(device)).shape)#.reshape(1,8)
#   break

In [0]:
# df2['inp']
labels = {0: 'Potential_Threat_Anxiety_',
 1: 'Frustrative_Nonreward',
 2: 'Sustained_Threat',
 3: 'Sleep_Wakefulness',
 4: 'Circadian_Rhythms',
 5: 'Arousal',
 6: 'Loss',
 7: 'Acute_Threat_Fear'}

In [0]:
gdf = df2.groupby('label')

In [0]:
# gdf.groups
construct_test = {}
for i in range(8):
  construct_test[i]= df2.iloc[gdf.groups[i]]

In [0]:
# Y_pred = y_pred.cpu().detach().numpy()

In [0]:
# Y_belongs = np.array(Y_belongs).reshape((len(Y_belongs),1))


In [0]:
# Y_test = np.array(Y_test)*np.

In [0]:
# np.concatenate((Y_pred,Y_belongs),axis =1).shape

In [0]:
# accuracy_score(Y_test, Y_pred)

In [0]:
def ap(y_pred,y_belong):
  y_pred = np.array(y_pred)
  y_belong = np.array(y_belong)
  y_pred = y_pred.reshape(-1,1)
  y_belong = y_belong.reshape(-1,1)
  arr = np.concatenate((y_pred,y_belong),axis=1)
  
  arr = sorted(arr,key = lambda x:x[0],reverse=True)
  
  ans = 0
  rel = 0
  num = 0
  for i in range(len(y_pred)):
    if arr[i][1] == 1:
      rel = rel+1
      ans += rel/(i+1)
      num = num+1
  return ans/num

def map(y_pred,y,y_belong):
  j = 0
  j_prev = 0
  ans = 0
  for i in range(8):
    while j<len(y) and y[j] == i:
      j = j+1
    ans = ans + ap(y_pred[j_prev:j],y_belong[j_prev:j])
    j_prev = j
  return ans/8


In [0]:
map_list = []
class_count = {}
for _ in range(1):
  aps = 0
  for cl_ in construct_test.keys():
    X_test = construct_test[cl_]['inp'].apply(toTensor).values
    y_pred = torch.randn((len(X_test),1)).to(device)
    class_count[cl_] = len(X_test)
    for i in range(len(X_test)):
      y_pred[i] = model(X_test[i].to(device))[cl_,0].reshape(1,1)
    y_pred = y_pred.cpu().detach().numpy()
    # print(y_pred.shape)
    y_belongs = np.array(construct_test[cl_]['y_belong'].values).reshape(len(X_test),1)
    # print(y_belongs.shape)
    # print(np.concatenate((y_belongs,y_pred),axis =1 ))
    t = ap(y_pred,y_belongs)
    print(labels[cl_],t)
    aps += t
  # println()
  map_list.append(aps/8)
  # break


In [0]:
print(class_count)

In [0]:

print(sum(map_list)/len(map_list))

In [0]:
len(list(model.parameters()))

In [0]:
for parameter in model.parameters():
  print(parameter.shape)